<a href="https://colab.research.google.com/github/Djadih/DataScience_Project/blob/main/midi_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo, meter
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
 from google.colab import files
uploaded = files.upload()

Saving mz_331_1.mid to mz_331_1 (2).mid


In [316]:
def get_notes(filename):
  notes = []
  offset = []
  midi = converter.parse(filename)
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append([element.pitch.midi])
      offset.append(element.offset)
    elif isinstance(element, chord.Chord):
      notes.append([n.midi for n in element.pitches])
      offset.append(element.offset)
  return notes, offset

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 400

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    i  = []
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, normalized_input, network_output)

In [ ]:
notes, offset = get_notes('beethoven_opus10_1.mid')

In [ ]:
len(offset)

2318

In [330]:
notes[:30]

[[63, 67, 72],
 [60, 55, 51, 48],
 [67],
 [75],
 [72],
 [79],
 [75],
 [84],
 [79],
 [87],
 [87],
 [67, 72],
 [60, 63],
 [67, 72],
 [63, 60],
 [67, 71],
 [62, 65],
 [68, 65, 71],
 [53, 59, 56, 62, 50],
 [71],
 [77],
 [74],
 [83],
 [77],
 [86],
 [83],
 [89],
 [89],
 [71, 74],
 [62, 68, 65]]

In [ ]:
def create_chords_with_offset(notes, offset):
  length = int(offset[-1]*4 + 1)
  chords = [ '' for i in range(length) ]
  for idx in range(len(notes)):
    chord_idx = int(offset[idx]*4)
    if len(chords[chord_idx]) == 0:
      if len(notes[idx]) == 1:
        chords[chord_idx] = str(notes[idx][0])
      else:
        new_chord = chord.Chord(notes[idx])
        chords[chord_idx] = '.'.join(str(n) for  n in new_chord.normalOrder) 
  return chords

In [ ]:
test_chords = create_chords_with_offset(notes, offset)

In [341]:
test_chords[:30]

['0.3.7',
 '',
 '',
 '',
 '',
 '',
 '',
 '67',
 '75',
 '',
 '',
 '72',
 '79',
 '',
 '',
 '75',
 '84',
 '',
 '',
 '79',
 '87',
 '',
 '',
 '',
 '87',
 '',
 '',
 '',
 '',
 '']

In [ ]:
n_vocab = len(set(test_chords))
n_vocab

101

In [317]:
network_input, normalized_input,  network_output = prepare_sequences(test_chords, n_vocab)

In [ ]:
network_input

In [318]:
normalized_input.shape

(4245, 400, 1)

In [319]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [320]:
model = create_network(normalized_input, n_vocab)

In [321]:
model.fit(normalized_input, network_output, epochs=500, batch_size=128)

Epoch 1/500
34/34 [==============================] - 11s 225ms/step - loss: 2.6481
Epoch 2/500
34/34 [==============================] - 8s 227ms/step - loss: 2.0738
Epoch 3/500
34/34 [==============================] - 8s 228ms/step - loss: 2.1492
Epoch 4/500
34/34 [==============================] - 8s 229ms/step - loss: 2.0783
Epoch 5/500
34/34 [==============================] - 8s 231ms/step - loss: 2.1025
Epoch 6/500
34/34 [==============================] - 8s 233ms/step - loss: 2.0088
Epoch 7/500
34/34 [==============================] - 8s 234ms/step - loss: 1.9141
Epoch 8/500
34/34 [==============================] - 8s 235ms/step - loss: 1.9103
Epoch 9/500
34/34 [==============================] - 8s 237ms/step - loss: 1.9087
Epoch 10/500
34/34 [==============================] - 8s 239ms/step - loss: 1.8266
Epoch 11/500
34/34 [==============================] - 8s 239ms/step - loss: 1.8445
Epoch 12/500
34/34 [==============================] - 8s 239ms/step - loss: 1.8636
Epoch 13/500

In [336]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = 0

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [337]:
pitchnames = sorted(set(item for item in test_chords))
output = generate_notes(model, network_input,pitchnames, n_vocab)

In [ ]:
output[:30]

In [339]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """

    output_notes = []

    # create note and chord objects based on the values generated by the model
    for idx, pattern in enumerate(prediction_output):
      if len(pattern) != 0:
        notes_in_chord = [int(n) for n in pattern.split('.')]
        new_chord = chord.Chord(notes_in_chord)
        new_chord.offset = idx/4
        output_notes.append(new_chord)
        # increase offset each iteration so that notes do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [340]:
create_midi(output)